In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

HOST_IP = os.environ['HOST_IP']
DATABASE_USER = os.environ['DATABASE_USER']
DATABASE_PASSWORD = os.environ['DATABASE_PASSWORD']
DATABASE_PORT = os.environ['DATABASE_PORT']

connection_url = URL.create(
    "postgresql+psycopg2",
    username=DATABASE_USER,
    password=DATABASE_PASSWORD,
    host=HOST_IP,
    port=DATABASE_PORT,
    database="mimiciv"
)

engine = create_engine(connection_url)

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
from sqlalchemy import text

query = text("SELECT * FROM mimiciv.mimiciv_hosp.admissions LIMIT 10")
result = session.execute(query)

for row in result:
    print(row)

In [ ]:
import pandas as pd

pd.read_sql_query(query, engine)